<center> <img src="https://storage.cloud.google.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="300"/> </center>

# <center>Getting Started with Hosted Phoenix

This guide demonstrates how to use hosted phoenix, our solution for AI developers to trace and evaluate their LLM applications without setting up infrastructure. Read more about Phoenix [here](https://docs.arize.com/phoenix/).

ℹ️ This notebook requires an OpenAI API key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies. It does NOT require installing arize-phoenix to log traces, as we are compliant with OpenTelemetry tracing.

In [1]:
# !pip install -q "arize-phoenix>=4.29.0" openinference-instrumentation-openai openai

In [2]:
# import os
# from getpass import getpass

# if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
#     openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

# os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
import os
from getpass import getpass

# if not (PHOENIX_API_KEY := os.getenv("PHOENIX_API_KEY")):
#     PHOENIX_API_KEY = getpass("🔑 Enter your Phoenix API key: ")
PHOENIX_API_KEY = "usertest:usertest"
os.environ["PHOENIX_API_KEY"] = PHOENIX_API_KEY

## Step 2: Setup Tracing
Let's send a trace to Hosted Phoenix!

In [4]:
# from openinference.instrumentation.openai import OpenAIInstrumentor

# from phoenix.otel import register

# os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
# os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

# # Setup OTEL tracing for hosted Phoenix. The register function will automatically detect the endpoint and headers from your environment variables.
# tracer_provider = register()

# # Turn on instrumentation for OpenAI
# OpenAIInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)

Send query to Hosted Phoenix


In [5]:
# import openai

# openai_client = openai.OpenAI()
# response = openai_client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[{"role": "user", "content": "Write a haiku."}],
#     max_tokens=20,
# )
# print(response.choices[0].message.content)

## Step 3: Access Phoenix Instance

From here, you can access your Phoenix instance to power evaluations, experiments, upload datasets, etc. The code below queries spans from the last 7 days, and exports them to a pandas dataframe.


In [6]:
from datetime import datetime, timedelta

import phoenix as px

# Initiate Phoenix client
# os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key=usertest:usertest"
# px_client = px.Client(endpoint="http://localhost:4041")
px_client = px.Client()

/Users/kikocastillo/anaconda3/envs/phoenix-dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
px.Client?

Init signature:
px.Client(
    *,
    endpoint: Optional[str] = None,
    warn_if_server_not_running: bool = True,
    headers: Optional[Mapping[str, str]] = None,
    api_key: Optional[str] = None,
    **kwargs: Any,
)
Docstring:     
An abstract base class intended to constraint both `Client` and
`Session` so that they both implement the same methods.
Init docstring:
Client for connecting to a Phoenix server.

Args:
    endpoint (str, optional): Phoenix server endpoint, e.g.
        http://localhost:6006. If not provided, the endpoint will be
        inferred from the environment variables.

    headers (Mapping[str, str], optional): Headers to include in each
        network request. If not provided, the headers will be inferred from
        the environment variables (if present).
File:           ~/Documents/Arize/phoenix/src/phoenix/session/client.py
Type:           ABCMeta
Subclasses:     

In [8]:
px_client.query_spans?

Signature:
px_client.query_spans(
    *queries: phoenix.trace.dsl.query.SpanQuery,
    start_time: Optional[datetime.datetime] = None,
    end_time: Optional[datetime.datetime] = None,
    limit: Optional[int] = 1000,
    root_spans_only: Optional[bool] = None,
    project_name: Optional[str] = None,
    stop_time: Optional[datetime.datetime] = None,
    timeout: Optional[int] = 5,
) -> Union[pandas.core.frame.DataFrame, List[pandas.core.frame.DataFrame], NoneType]
Docstring:
Queries spans from the Phoenix server or active session based on specified criteria.

Args:
    queries (SpanQuery): One or more SpanQuery objects defining the query criteria.
    start_time (datetime, optional): The start time for the query range. Default None.
    end_time (datetime, optional): The end time for the query range. Default None.
    root_spans_only (bool, optional): If True, only root spans are returned. Default None.
    project_name (str, optional): The project name to query spans for. This can be

In [9]:
# Get spans from the last 7 days only
start = datetime.now() - timedelta(days=70)

# Get spans to exclude the last 24 hours
end = datetime.now() 

phoenix_df = px_client.query_spans(project_name="demo_llama_index",timeout=60)
# phoenix_df = px_client.get_trace_dataset()
# phoenix_df.head()

query_spans from session/client
response=<Response [200 OK]>
type(response.content)=<class 'bytes'>


In [10]:
phoenix_df

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id,...,attributes.reranker.output_documents,attributes.llm.model_name,attributes.llm.token_count.completion,attributes.llm.output_messages,attributes.llm.input_messages,attributes.llm.token_count.total,attributes.llm.token_count.prompt,attributes.llm.invocation_parameters,attributes.llm.prompt_template.variables,attributes.llm.prompt_template.template
context.span_id,,,,,,,,,,,,,,,,,,,,,
2ce2955e565ebc9f,OpenAIEmbedding._get_query_embedding,EMBEDDING,3e7db7106a63f929,2024-10-15 00:40:39.536429+00:00,2024-10-15 00:40:39.740309+00:00,OK,,[],2ce2955e565ebc9f,cfb324efd848b7fa73d41be085adfef4,...,None,None,NaN,None,None,NaN,NaN,None,None,None
3e7db7106a63f929,BaseEmbedding.get_query_embedding,EMBEDDING,ed4ed7178f41f93c,2024-10-15 00:40:39.534877+00:00,2024-10-15 00:40:39.749646+00:00,OK,,[],3e7db7106a63f929,cfb324efd848b7fa73d41be085adfef4,...,None,None,NaN,None,None,NaN,NaN,None,None,None
ed4ed7178f41f93c,VectorIndexRetriever._retrieve,RETRIEVER,d46e0b4db23c8abb,2024-10-15 00:40:39.534249+00:00,2024-10-15 00:40:39.792803+00:00,OK,,[],ed4ed7178f41f93c,cfb324efd848b7fa73d41be085adfef4,...,None,None,NaN,None,None,NaN,NaN,None,None,None
d46e0b4db23c8abb,BaseRetriever.retrieve,RETRIEVER,4ffa8711fa109514,2024-10-15 00:40:39.533047+00:00,2024-10-15 00:40:39.797856+00:00,OK,,[],d46e0b4db23c8abb,cfb324efd848b7fa73d41be085adfef4,...,None,None,NaN,None,None,NaN,NaN,None,None,None
3b232e67ef0d5d19,CohereRerank._postprocess_nodes,RERANKER,4ffa8711fa109514,2024-10-15 00:40:39.801447+00:00,2024-10-15 00:40:40.043641+00:00,OK,,[],3b232e67ef0d5d19,cfb324efd848b7fa73d41be085adfef4,...,[{'document.content': 'Setup Tracing Phoenix u...,None,NaN,None,None,NaN,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b1e705e2a4eee3e1,TokenTextSplitter.split_text,CHAIN,0dddde8bf03aa61c,2024-10-15 00:42:22.317587+00:00,2024-10-15 00:42:22.321592+00:00,OK,,[],b1e705e2a4eee3e1,5eb9b6e54c1df4c9b8987eda5a736e0e,...,None,None,NaN,None,None,NaN,NaN,None,None,None
f60bff8e5c54c070,TokenTextSplitter.split_text,CHAIN,371216b07c870462,2024-10-15 00:42:22.330372+00:00,2024-10-15 00:42:22.332835+00:00,OK,,[],f60bff8e5c54c070,5eb9b6e54c1df4c9b8987eda5a736e0e,...,None,None,NaN,None,None,NaN,NaN,None,None,None
5436c9c3bf6e1007,OpenAI.chat,LLM,072870edc8ee7ff6,2024-10-15 00:42:22.338902+00:00,2024-10-15 00:42:23.676235+00:00,OK,,[],5436c9c3bf6e1007,5eb9b6e54c1df4c9b8987eda5a736e0e,...,None,gpt-3.5-turbo,112.0,[{'message.content': 'To set up automatic inst...,[{'message.content': 'You are an expert Q&A sy...,982.0,870.0,"{""context_window"": 16384, ""num_output"": -1, ""i...",None,None


In [11]:
# import pandas as pd
# df = pd.DataFrame(
#     [
#         {
#             "question": "What is Paul Graham known for?",
#             "answer": "Co-founding Y Combinator and writing on startups and techology.",
#             "metadata": {"topic": "tech"},
#         }
#     ]
# )
# # phoenix_client = px.Client()
# dataset = px_client.upload_dataset(
#     dataframe=df,
#     dataset_name="test-dataset",
#     input_keys=["question"],
#     output_keys=["answer"],
#     metadata_keys=["metadata"],
# )


In [12]:
# ds = px_client.get_dataset(name="test-dataset")
# ds

In [13]:
# ds.as_dataframe()

In [14]:
# evals = px_client.get_evaluations(project_name="demo_llama_index")

In [15]:
# evals

In [16]:
df = px_client.get_spans_dataframe(project_name="demo_llama_index")

get_spans_dataframe from data_extractor.py
query_spans from session/client
response=<Response [200 OK]>
type(response.content)=<class 'bytes'>
